In [ ]:
# !pip install tensorflow-gpu==2.0.0.alpha0 tensorflow==2.0.0 pandas-datareader

     |███████████████████████▋        | 245.1MB 6.9MB/s eta 0:00:133    |█████▊                          | 59.8MB 16.4MB/s eta 0:00:17     |██████▏                         | 63.7MB 16.4MB/s eta 0:00:17

In [1]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from empyrical import roll_max_drawdown, alpha, beta
from sklearn.preprocessing import MinMaxScaler
from src.asset_selector import AssetSelector
from src.indicator_collection import IndicatorCollection as Indicators
from util import time_formatter, df2csv
from src.predictor import Predictor
from sklearn.cluster import KMeans
from tqdm import tqdm_notebook, tqdm
from collections import deque
import pandas_datareader as data_reader
import alpaca_trade_api as tradeapi
import matplotlib.pyplot as plt
import plotly.graph_objs as go
import plotly.offline as py
import tensorflow as tf
import pandas as pd
import numpy as np
import configparser
import requests
import random
import json
import time
import sys
import os
import math

tf.__version__

'2.0.0'

In [7]:
# start here by entering a ticker symbol
ticker = "ACOR"
# number of weeks to look back
weeks = 54
# and watch the magic happen below...

backdate = time_formatter(time.time() - (604800 * weeks), time_format="%Y-%m-%d")
config = configparser.ConfigParser()
try:
    config.read(os.path.relpath("../config.ini"))
except FileExistsError as e:
    print("FileExistsError: {}".format(e))
    sys.exit(1)
alpaca_api = tradeapi.REST(
    base_url    = config["alpaca"]["APCA_API_BASE_URL"],
    key_id      = config["alpaca"]["APCA_API_KEY_ID"],
    secret_key  = config["alpaca"]["APCA_API_SECRET_KEY"],
    api_version = config["alpaca"]["VERSION"]
)
trading_account = alpaca_api.get_account()

selector            = AssetSelector(alpaca_api, edgar_token=None)
indicators          = Indicators(alpaca_api)
scaler              = MinMaxScaler()
day_bars            = selector.get_barset(ticker, "day", backdate)
day_bars            = selector.extract_bar_data(day_bars, ticker)
minute_bars         = selector.get_barset(ticker, "minute", backdate)
minute_bars         = selector.extract_bar_data(minute_bars, ticker)
day_spy_bars        = selector.get_barset("SPY", "1D", backdate)
day_spy_bars        = selector.extract_bar_data(day_spy_bars, "SPY")
minute_spy_bars     = selector.get_barset("SPY", "minute", backdate)
minute_spy_bars     = selector.extract_bar_data(minute_spy_bars, "SPY")
ticker_cluster      = indicators.get_ticker_indicators(ticker)
ticker_cluster["spy_day_close"] = day_spy_bars["close"]

sleep 3 seconds and retrying https://data.alpaca.markets/v1/bars/1D 3 more time(s)...


[Debug] Backdate debug get_ticker_indicators


/home/ben/anaconda3/envs/u-nectix/lib/python3.7/site-packages/finta/finta.py:263: FutureWarning:

Currently, 'apply' passes the values as ndarrays to the applied function. In the future, this will change to passing it as Series objects. You need to specify 'raw=True' to keep the current behaviour, and you can pass 'raw=False' to silence this warning



In [8]:
ticker_cluster["close"].head()

2019-09-06 00:00:00-04:00    2.80
2019-09-09 00:00:00-04:00    3.15
2019-09-10 00:00:00-04:00    4.01
2019-09-11 00:00:00-04:00    4.01
2019-09-12 00:00:00-04:00    3.85
Name: close, dtype: float64

In [9]:
def sigmoid(num):
  """Return the sigmoid value of num.
  
  :return: sigmoid value
  """
  return 1 / (1 + math.exp(-num))

def price_format(num): 
    """Print the price with proper decimal, positive and negative formatting.  
    
    :return: formatted stock price
    """
    if num < 0:
        return "- ${0:2f}".format(abs(num))
    else:
        return "${0:2f}".format(abs(num))

def load_dataset(ticker, start):
  """Load the dataset for a given ticker symbol.
  
  :param ticker: a stock ticker symbol
  :param start: 
  :return close: return the most recent closing price in the time series
  """
  dataset = data_reader.DataReader(ticker, start=start, data_source="yahoo")
  start_date = str(dataset.index[0]).split()[0]
  end_date = str(dataset.index[-1]).split()[0]
  close = dataset["Close"]
  return close

def create_state(data, step, window_size):
  """Create state (duh).

  :param data:
  :param step:
  :param window_size:
  :return :
  """
  starting_id = step - window_size + 1

  if starting_id >= 0:
    windowed_data = data[starting_id:step+1]
  else:
    windowed_data =  -starting_id * [data[0]] + list(data[0:step+1])

  state = []
  for i in range(window_size - 1):
    state.append(sigmoid(windowed_data[i+1] - windowed_data[i]))
  return np.array([state])

In [10]:
class Qtrader():

  def __init__(self, state_size, action_space=3, model_name="Qtrader"):
    """Initialize the AI trader class.
    
    :param state_size:
    :param action_space: stay in position, buy, sell
    :param model_name:
    """
    self.state_size = state_size
    self.action_space = action_space
    self.memory = deque(maxlen=2000)
    self.inventory = []
    self.model_name = model_name 

    # maximize current reward over long term rewards
    self.gamma = 0.95

    # should we use a random action or let the model choose?
    # in beginning, this will be random and will taper off as the system learns
    self.epsilon = 1.0
    
    # when epison == this, stop decresing 
    self.epsilon_final = 0.01
    
    # how fast should the epsilon decrease?
    self.epsilon_decay = 0.995

    # initialize the model
    self.model = self.model_builder()

  def model_builder(self):
    """Build our model."""

    model = tf.keras.models.Sequential()

    # first dense layer
    model.add(tf.keras.layers.Dense(units=32, activation="relu", input_dim=self.state_size))

    # second layer
    model.add(tf.keras.layers.Dense(units=64, activation="relu"))

    # third layer
    model.add(tf.keras.layers.Dense(units=128, activation="relu"))

    # output layer
    model.add(tf.keras.layers.Dense(units=self.action_space, activation="linear"))

    # compile the model
    model.compile(loss="mse", optimizer=tf.keras.optimizers.Adam(lr=0.001))

    return model
  
  def trade(self, state):
    """Determine if action should be random or from the model and trade.
    
    :param state: 
    :return: 
    """
    # if the random number is less than the epsilon value, act randomly
    if random.random() <= self.epsilon:
      return random.randrange(self.action_space)

    # make a prediction given a state argument
    actions = self.model.predict(state)
    return np.argmax(actions[0])

  def batch_trade(self, batch_size):
    """Loop through and execute all trades in a batch.

    :param batch_size: the number of trades to make
    """
    batch = []

    # iterate through deque memory
    for i in range(len(self.memory) - batch_size + 1, len(self.memory)):
      # append value from the end
      batch.append(self.memory[i])
    
    # train the model
    for state, action, reward, next_state, done in batch:
      # state, action, reward, next_state, done
      reward = reward

      # terminal (done) state?
      if not done:
        # calculate reward
        reward = reward + self.gamma * np.amax(self.model.predict(next_state)[0])
      
      # set a target variable
      target = self.model.predict(state)

      target[0][action] = reward

      # fit the model now that we have target and state
      self.model.fit(state,target,epochs=1,verbose=0)

    # decrease epsilon by epsilon_decay value
    if self.epsilon > self.epsilon_final:
      self.epsilon *= self.epsilon_decay


In [11]:
# ticker = "AAPL"
data = load_dataset(ticker, backdate)
# data = ticker_cluster["close"]
data.head()

Date
2018-11-23    19.049999
2018-11-26    20.280001
2018-11-27    19.870001
2018-11-28    20.809999
2018-11-29    20.129999
Name: Close, dtype: float64

In [15]:
# Some hyperparameters to play with
window_size = 10
episodes = 100
batch_size = 32
data_samples = len(data) - 1

In [16]:
trader = Qtrader(window_size)

In [17]:
trader.model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 32)                352       
_________________________________________________________________
dense_5 (Dense)              (None, 64)                2112      
_________________________________________________________________
dense_6 (Dense)              (None, 128)               8320      
_________________________________________________________________
dense_7 (Dense)              (None, 3)                 387       
Total params: 11,171
Trainable params: 11,171
Non-trainable params: 0
_________________________________________________________________


In [18]:
def run():
  # do the thang
  for episode in range(episodes + 1):
    print("Episode\t {}/{}".format(episode, episodes))
    state = create_state(data, 0, window_size+1)
    total_profit = 0
    trader.inventory = []
    for t in tqdm(range(data_samples)):
      action = trader.trade(state)
      next_state = create_state(data, t+1, window_size+1)
      reward = 0

      if action == 1:     # buy
        trader.inventory.append(data[t])
      
      elif action == 2 and len(trader.inventory) > 0:   # sell
        buy_price = trader.inventory.pop(0)
        reward = max(data[t] - buy_price, 0)
        total_profit += data[t] - buy_price

      if t == data_samples - 1:
        done = True
      else:
        done = False

      trader.memory.append([state, action, reward, next_state, done])
      state = next_state

      if done:
        print("Total profit:\t".ljust(10), str(total_profit))
      
      if len(trader.memory) > batch_size:
        trader.batch_trade(batch_size)
    if episode %10 == 0:
      trader.model.save("Qtrader_{}.h5".format(episode))

In [1]:
run()

ERROR:root:File `'().py'` not found.
